**Funciones**

Luego de haber realizazo el modelado previo en nuestro archivo 'PI_ML_OPS_Pablo_v3.ipynb' crearemos las funciones que utilizará nuestra API.

In [34]:
import pandas as pd
import numpy as np

Comenzaremos creando un nuevo DataFrame con el archivo .json que exportamos en el archivo previo.

In [35]:
df_search = pd.read_json('search_data.json')


Realizamos un vista rápida.

In [36]:
df_search.head()

,genres,title,release_date,specs,early_access,sentiment,metascore
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],False,None,None
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",False,Mostly Positive,None
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",False,Mostly Positive,None
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,[Single-player],False,None,None
4,None,None,None,"[Single-player, Full controller support, HTC V...",False,None,None


Verifiquemos la info del set

In [37]:
df_search.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28852 non-null  object
 1   title         30085 non-null  object
 2   release_date  29894 non-null  object
 3   specs         31465 non-null  object
 4   early_access  32135 non-null  bool  
 5   sentiment     24953 non-null  object
 6   metascore     2677 non-null   object
dtypes: bool(1), object(6)
memory usage: 1.7+ MB


Hacemos una limpieza y preparado de los datos antes de continuar trabajando con ellos.

In [38]:
df_search['release_date'] = pd.to_datetime(df_search["release_date"], errors='coerce')
df_search['metascore'] = pd.to_numeric(df_search['metascore'], errors='coerce')
df_search.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28852 non-null  object        
 1   title         30085 non-null  object        
 2   release_date  29894 non-null  datetime64[ns]
 3   specs         31465 non-null  object        
 4   early_access  32135 non-null  bool          
 5   sentiment     24953 non-null  object        
 6   metascore     2607 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(1), object(4)
memory usage: 1.7+ MB


**`CONSIGNA`**:

`Desarrollo API`: Propones disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas que propones son las siguientes:

Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

+ def *genero( *`Año`: str* )*:
    Se ingresa un año y devuelve una lista con los 5 géneros más vendidos en el orden correspondiente.

+ def *juegos( *`Año`: str* )*:
    Se ingresa un año y devuelve una lista con los juegos lanzados en el año.

+ def *specs( *`Año`: str* )*:
    Se ingresa un año y devuelve una lista con los 5 specs que más se repiten en el mismo en el orden correspondiente. 

+ def *earlyacces( *`Año`: str* )*:
    Cantidad de juegos lanzados en un año con early access.

+ def *sentiment( *`Año`: str* )*:
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros que se encuentren categorizados con un análisis de sentimiento. 

    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ejemplo de retorno: *{Mixed = 182, Very Positive = 120, Positive = 278}*

+ def *metascore( *`Año`: str* )*:
    Top 5 juegos según año con mayor metascore.

**FUNCIONES**

Crearemos una serie de funciones que nos brindaran información específica sobre el conjunto de datos de juegos en función del año proporcionado. Esto podrá ser útil para realizar análisis y visualizaciones más detalladas eventualmente.

In [39]:
def find_year(anio):
    """Esta función sirve de soporte para otras funciones. Al recibir un año (int),
       devuelve un DataFrame que contiene únicamente los datos correspondientes a ese año."""
    df_anio = df_search[df_search['release_date'].dt.year == anio]
    return df_anio


def genre(anio):
    """Al recibir un año (int) devuelve una lista con los 5 géneros más vendidos en el orden requerido."""
    df_genre = find_year(anio)
    df_genre = df_genre.explode("genres")
    lista_generos = df_genre['genres'].value_counts().head().index.to_list()
    return lista_generos


def games(anio):
    """Al recibir un año (int) devuelve una lista con los juegos lanzados en el año."""
    df_games = find_year(anio)
    lista_games = df_games.title.to_list()
    return lista_games


def specs(anio):
    """Al recibir un año (int) devuelve una lista con las 5 specs más repetidas en el orden correspondiente."""
    df_specs = find_year(anio)
    df_specs = df_specs.explode("specs")
    lista_specs = df_specs['specs'].value_counts().head().index.to_list()
    return lista_specs


def earlyacces(anio):
    """Al recibir un año (int) devuelve la cantidad de juegos lanzados en ese año con early access."""
    df_early = find_year(2019)
    early = df_early['early_access'].sum()
    return early


def sentiment(anio):
    """Al recibir un año (int) devuelve una lista con la cantidad de registros que
       se encuentren categorizados con un análisis de sentimiento ese año."""
    df_sentiment = find_year(anio)
    sent_on = (df_sentiment["sentiment"] == 'Overwhelmingly Negative').sum()
    sent_vn = (df_sentiment["sentiment"] == 'Very Negative').sum()
    sent_n  = (df_sentiment["sentiment"] == 'Negative').sum()
    sent_mn = (df_sentiment["sentiment"] == 'Mostly Negative').sum()
    sent_m  = (df_sentiment["sentiment"] == 'Mixed').sum()
    sent_mp = (df_sentiment["sentiment"] == 'Mostly Positive').sum()
    sent_p  = (df_sentiment["sentiment"] == 'Positive').sum()
    sent_vp = (df_sentiment["sentiment"] == 'Very Positive').sum()
    sent_op = (df_sentiment["sentiment"] == 'Overwhelmingly Positive').sum()

    sent_on_str = f"Overwhelmingly Negative: {sent_on}"
    sent_vn_str = f"Very Negative: {sent_vn}"
    sent_n_str  = f"Negative: {sent_n}"
    sent_mn_str = f"Mostly Negative: {sent_mn}"
    sent_m_str  = f"Mixed: {sent_m}"
    sent_mp_str = f"Mostly Positive: {sent_mp}"
    sent_p_str  = f"Positive: {sent_p}"
    sent_vp_str = f"Very Positive: {sent_vp}"
    sent_op_str = f"Overwhelmingly Positive: {sent_op}"

    lista = [[sent_on, sent_on_str], [sent_vn, sent_vn_str], [sent_n, sent_n_str], [sent_mn, sent_mn_str], [sent_m, sent_m_str],
             [sent_mp, sent_mp_str], [sent_p, sent_p_str], [sent_vp, sent_vp_str], [sent_op, sent_op_str]]

    lista_final = []

    for sent in lista:
        if sent[0] > 0:
            lista_final.append(sent[1])

    return lista_final


def metascore(anio):
    """Al recibir un año (int) devuelve  el top 5 juegos con mayor metascore."""
    df_meta = find_year(anio)
    df_meta = df_meta[['title', 'metascore']].sort_values('metascore', axis=0, ascending=False).head()

    lista_name_score = []

    for i in range(df_meta.shape[0]):
        name = df_meta.iloc[i:i+1, 0:1].values[0][0]
        score = df_meta.iloc[i:i+1, 1:2].values[0][0]
        name_score = f"{name}: {score}"
        lista_name_score.append(name_score)
    return lista_name_score

Buscamos los valores únicos en la columna "sentiment", para comprender qué tipos de categorías de sentimiento están presentes en los datos y qué tipos de análisis de sentimiento se han realizado en los juegos.

In [40]:
df_search.sentiment.unique()

array([None, 'Mostly Positive', 'Mixed', '1 user reviews',
       '3 user reviews', '8 user reviews', 'Very Positive',
       'Overwhelmingly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Very Negative', 'Positive', 'Mostly Negative',
       '9 user reviews', 'Negative', '4 user reviews', '7 user reviews',
       'Overwhelmingly Negative'], dtype=object)

- 'Overwhelmingly Negative'
- 'Very Negative'
- 'Negative'
- 'Mostly Negative'
- 'Mixed'
- 'Mostly Positive'
- 'Positive'
- 'Very Positive'
- 'Overwhelmingly Positive'

A continuación hacemos un recuento de cada valor único en la columna 'sentiment' y los ordenamos de mayor a menor.

In [41]:
df_search.groupby('sentiment')['sentiment'].count().sort_values(ascending=False)

sentiment
Mixed                      4103
Very Positive              3868
Positive                   3281
Mostly Positive            2744
1 user reviews             2496
2 user reviews             1756
3 user reviews             1231
4 user reviews              964
5 user reviews              846
Mostly Negative             802
6 user reviews              756
7 user reviews              619
8 user reviews              537
9 user reviews              488
Overwhelmingly Positive     303
Negative                    123
Very Negative                29
Overwhelmingly Negative       7
Name: sentiment, dtype: int64

A continuación visualizamos una lista de los 50 juegos más recientes según su fecha de lanzamiento.

In [42]:
df_search.sort_values('release_date', ascending=False).head(50)

,genres,title,release_date,specs,early_access,sentiment,metascore
13140,"[Adventure, Indie, RPG]",Finding Paradise Soundtrack,2021-12-31,"[Single-player, Downloadable Content]",False,Positive,NaN
14432,"[Action, Adventure, Indie, RPG, Strategy, Earl...",The Legendary Player - Make Your Reputation - ...,2019-12-10,"[Single-player, Steam Achievements]",True,3 user reviews,NaN
21368,[RPG],The End of an Age: Fading Remnants,2019-09-16,"[Single-player, Full controller support]",False,None,NaN
14221,"[Action, Adventure, Indie]",Raji: An Ancient Epic,2019-05-01,"[Single-player, Steam Achievements, Full contr...",False,None,NaN
14328,"[Adventure, Free to Play, Indie]",K'NOSSOS,2019-01-01,[Single-player],False,None,NaN
11773,"[Adventure, Early Access]",Reality Incognita,2018-12-31,"[Single-player, Steam Achievements]",True,None,NaN
13106,[Adventure],Nightshade Additional Scenarios,2018-12-20,"[Single-player, Downloadable Content, Steam Ac...",False,3 user reviews,NaN
11737,"[Action, Adventure, Indie, RPG, Simulation, St...",INTERSTELLAR PRIME,2018-12-17,"[Single-player, Full controller support, Capti...",True,None,NaN
13641,[Adventure],Welcome to Orochi Park,2018-12-03,[Single-player],False,None,NaN
20109,"[Action, Indie, Simulation, Early Access]",INTRUDER - WAR AREAS,2018-12-01,[Single-player],True,6 user reviews,NaN
